In [68]:
"""
D17 과제
- 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
- 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
- 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
- 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

작업 순서
1. memory 없이 먼저 구현. d16 기준
2. 강의 듣고, memory 구현

"""

'\nD17 과제\n- 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨\u200d✈️🔥". "대부" -> "👨\u200d👨\u200d👦🔫🍝").\n- 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.\n- 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.\n- 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.\n\n작업 순서\n1. memory 없이 먼저 구현. d16 기준\n2. 강의 듣고, memory 구현\n\n'

In [70]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough


chat = ChatOpenAI(
    temperature=0.5,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=1000,
    return_messages=True,
    memory_key="chat_history",
)

examples = [
    {
        "movie": "스타워즈",
        "answer": """
        🌌⚔️👽

        - 🌌: 스타워즈는 우주 공간을 배경으로 한 환상적인 세계를 보여주는 영화이기 때문에 은은한 우주의 느낌을 담은 이모티콘을 선택했습니다.

        - ⚔️: 영화에서 빛나는 레이저 검이나 전투 장면이 많이 등장하는데, 이를 상징하는 이모티콘으로 스타워즈의 전투적인 요소를 표현했습니다.

    
        - 👽: 스타워즈는 다양한 외계 생명체들이 등장하는데, 그 중에서도 다양한 외계인들이 등장하여 영화의 다양성을 나타내기 때문에 외계 생명체를 상징하는 이모티콘을 선택했습니다.
        """,
    },
    {
        "movie": "탑건",
        "answer": """
        🚀🔥😎
        
        🚀: '탑건'은 전투기 조종사들의 역동적인 비행씬과 스릴 넘치는 전투 장면으로 인해 마치 하늘을 나는 듯한 느낌을 주는 영화입니다. 전투기가 속도를 내며 하늘을 가로지르는 장면은 마치 로켓처럼 빠르고 감동적입니다.

        🔥: 영화 속 주인공들의 열정과 용기가 불길처럼 뜨거운 에너지를 전달해줍니다. 전투기 조종사들이 마주하는 위험과 도전 속에서도 그들의 열정과 용기가 불을 지펴내는 듯한 느낌을 줍니다.

        😎: '탑건'은 스타일리시한 비행복과 섹시한 룩으로 유명한 영화로, 주인공들의 카리스마 넘치는 매력이 관객을 사로잡습니다. 영화 속 주인공들은 마치 쿨한 선장처럼 매 순간을 멋지게 소화해내는 것이 인상적입니다.
        """,
    },
    {
        "movie": "반지의 제왕",
        "answer": """
        👑🧝‍♂️🔥

        👑: '반지의 제왕'은 왕좌를 차지할 수 있는 힘을 상징하는 반지를 중심으로 전쟁과 운명의 이야기를 그린 영화로, 왕권과 권력에 대한 욕망을 상징하는 왕관을 상징합니다.

        🧝‍♂️: 영화 속에 등장하는 엘프들은 우아하고 아름다운 존재로, 자연과 조화를 이루며 숲을 수호하는 역할을 합니다. 엘프는 영화의 신비로움과 순수함을 상징합니다.

        🔥: 영화는 전투와 결투로 가득 차 있으며, 화려한 전투씬은 영화의 긴장감과 스릴을 높여줍니다. 불꽃같이 타오르는 전투는 캐릭터들의 용기와 희생을 상징합니다.        
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "영화 {movie} 에 대해서 이모티콘 3개로 표현해줘. 그리고 그 이유에 대해서 알려줘.",
        ),
        (
            "ai",
            "{answer}",
        ),
    ]
)
example_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 영화의 내용을 {style} 표현하는 것으로 유명한 {role} 입니다. ",
        ),
        example_prompt,
        MessagesPlaceholder(variable_name="chat_history"),
        (
            "human",
            "영화 {movie} 에 대해서 이모티콘 3개로 표현해줘. 그리고 그 이유에 대해서 알려줘.",
        ),
        # ("ai", "{answer}"),
    ]
)

general_prompt = ChatPromptTemplate.from_messages(
    [
        example_prompt,
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


movie_chain = RunnablePassthrough.assign(chat_history=load_memory) | final_prompt | chat
general_chain = (
    RunnablePassthrough.assign(chat_history=load_memory) | general_prompt | chat
)


def invoke_movie_chain(movie):
    result = movie_chain.invoke(
        {
            "role": "웹툰 작가",
            "style": "심미적으로",
            "movie": movie,
        }
    )
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )


def invoke_general_chain(question):
    result = general_chain.invoke(
        {
            "question": question,
        }
    )
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )


invoke_movie_chain("인생은 아름다워")

ValueError: Single '}' encountered in format string

In [ ]:
invoke_movie_chain("고질라")

🦖🔥🏙️

🦖: '고질라'는 거대한 괴물인 고질라를 상징하는 이모티콘입니다. 영화 속 고질라는 인류에게 위협을 가하는 거대하고 강력한 존재로, 이를 나타내는 이모티콘입니다.

🔥: 영화 속 고질라는 불길을 토해내며 도시를 파괴하는 장면이 인상적입니다. 고질라가 토해내는 불길은 파괴와 재앙을 상징하며, 영화의 긴장감과 스릴을 높여줍니다.

🏙️: 도시를 배경으로 한 '고질라'는 거대한 괴물과 인간들의 사투를 그린 영화로, 도시의 높은 건물들과 현대 문명을 상징하는 이모티콘입니다. 고질라와 도시의 대결은 인류의 힘과 무력함을 보여줍니다.

In [ ]:
memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='인생은 아름다워'),
 AIMessage(content="\n        🎨👨\u200d👦\u200d👦😊\n\n        🎨: '인생은 아름다워'는 삶의 아름다움과 가치를 담은 영화로, 삶의 다양한 색채와 아름다움을 상징하는 이모티콘입니다. 영화 속 아름다운 풍경과 감동적인 순간들이 관객들에게 희망과 용기를 전달합니다.\n\n        👨\u200d👦\u200d👦: 영화는 가족과 사랑에 대한 이야기를 중심으로 전개됩니다. 주인공과 가족들 간의 따뜻한 인연과 사랑이 영화의 중요한 요소이며, 이를 나타내는 이모티콘입니다.\n\n        😊: '인생은 아름다워'는 삶의 가치와 소중함을 일깨워주는 영화로, 희망과 긍정적인 마음가짐을 전하는 내용을 담고 있습니다. 이로 인해 관객들은 영화를 보고 나서 행복한 미소를 지을 수 있습니다.        \n        "),
 HumanMessage(content='고질라'),
 AIMessage(content="🦖🔥🏙️\n\n🦖: '고질라'는 거대한 괴물인 고질라를 상징하는 이모티콘입니다. 영화 속 고질라는 인류에게 위협을 가하는 거대하고 강력한 존재로, 이를 나타내는 이모티콘입니다.\n\n🔥: 영화 속 고질라는 불길을 토해내며 도시를 파괴하는 장면이 인상적입니다. 고질라가 토해내는 불길은 파괴와 재앙을 상징하며, 영화의 긴장감과 스릴을 높여줍니다.\n\n🏙️: 도시를 배경으로 한 '고질라'는 거대한 괴물과 인간들의 사투를 그린 영화로, 도시의 높은 건물들과 현대 문명을 상징하는 이모티콘입니다. 고질라와 도시의 대결은 인류의 힘과 무력함을 보여줍니다.")]

In [ ]:
invoke_general_chain("이전에 물어본 영화 제목이 무엇인가요?")

죄송합니다. 이전 질문에 대한 정보를 기억하지 못합니다. 다시 한 번 질문을 해주시겠습니까?